In [ ]:
let leverage ratio = (1.0M / (ratio - 1.0M)) + 1.0M
let price collateral debt l = (l * debt) / (collateral * (l - 1.0M))
let k collateral debt l = (debt * collateral) / (l * (l - 1.0M)) // in complete flux

let debtOut ratio collateral debt collateralIn = 
    let l = leverage ratio
    let currentPrice = price collateral debt l
    let excessCollateral = collateral - debt/currentPrice   
    let kStart = k collateral debt l
    //let out = debt - (kStart * (l - 1.0M) * l) / (collateral + collateralIn)

    // o = d - (k (l - 1) l)/(c + i) and c + i!=0 and l^2 !=l
    //let out = debt - (kStart * (l - 1.0M) * l) / (collateral + collateralIn)  
    // o = (c d + l (d t + k (-l) + k))/((l - 1) (c + l t)) and (l - 1) (c + l t)!=0 and l!=0
    //let out = (collateral * debt + (l * (debt * collateralIn - kStart * l + kStart))) / ((l - 1.0m) * (collateral + l * collateralIn))
    // o = k/(l^2 (c + t)) - d and l (c + t)!=0
    // let out = kStart / (l * l * (collateral + collateralIn)) - debt
    // // o = k/((l - 1) l (c + t)) - d and (l - 1) l (c + t)!=0
    // let out = kStart / ((l - 1.0M) * l * (collateral + collateralIn)) - debt
    // // o = d - k/((l - 1) l (c + t)) and (l - 1) l (c + t)!=0
    // let out = debt - kStart / ((l - 1.0M) * l * (collateral + collateralIn))
    // // o = d - k/((l - 1) l^2 (c + t)) and (l - 1) l (c + t)!=0
    // let out = debt - kStart / ((l - 1.0M) * l * l * (collateral + collateralIn))
    // // o = (c d (l - 1) + d (l - 1) t - k)/((l - 1) (c + t)) and (l - 1) (c + t)!=0 and l!=0
    // let out = (collateral * debt * (l - 1.0M) + debt * (l - 1.0M) * collateralIn - kStart) / ((l - 1.0M) * (collateral + collateralIn))
    // o = (c d + d t - k)/(c + t) and c + t!=0
    // let out = (collateral * debt + debt * collateralIn - kStart) / (collateral + collateralIn)
    // o = d - k/(l (c + t)) and l (c + t)!=0
    // let out = debt - kStart / (l * (collateral + collateralIn))
    // o = c - k/(d + t) and d + t!=0
    // let out = collateral - kStart / (debt + collateralIn)
    // o = c - (k (l - 1) l)/(d + t) and d + t!=0 and l^2 !=l
    let out = debt - (kStart * (l - 1.0M) * l) / (collateral + collateralIn)   
    // o = (c d + d t - k (l - 1) l)/(c + t) and c + t!=0 and l^2 !=l
    let out = (collateral * debt + debt * collateralIn - kStart * (l - 1.0M) * l) / (collateral + collateralIn)
    // o = (c d + d t - k)/(c + t) and c + t!=0
    let out = (collateral * debt + debt * collateralIn - kStart) / (collateral + collateralIn)

    let newCollateral = collateral + collateralIn
    let newDebt = debt + l * out
    let newPrice = price newCollateral newDebt l
    let newExcessCollateral = newCollateral - newDebt/newPrice
    let newRatio = (newPrice * newCollateral) / newDebt
    let newL = leverage newRatio
    let newK = k newCollateral (newCollateral * newPrice - newDebt) newL

    {| a = 0
    ; oldL = l
    ; oldK = k collateral debt l
    ; oldPrice = currentPrice
    ; oldExcessCollateral = excessCollateral
    ; collateralIn = collateralIn
    ; debtOut = out * l
    ; newEffectivePrice = out * l / collateralIn
    ; newCollateral = newCollateral
    ; newExcessCollateral = newExcessCollateral
    ; newDebt = newDebt
    ; newL = newL
    ; newK = newK
    ; newPrice = newPrice
    ; newRatio = newRatio
    |}

//let o = 
debtOut 2.0M 2.0M 1000.0M 0.1M
//printfn "debtOut: %A" o

a,collateralIn,debtOut,newCollateral,newDebt,newEffectivePrice,newExcessCollateral,newK,newL,newPrice,newRatio,oldExcessCollateral,oldK,oldL,oldPrice
0,0.1,95.23809523809523809523809524,2.1,1095.2380952380952380952380952,952.3809523809523809523809524,1.05,1150,2.0,1043.0839002267573696145124716,2,1.0,1000,2.0,1000


In [ ]:
type LSP = 
    { collateral : float 
    ; debt : float
    ; targetLeverage : float } 

let lsp = 
    { collateral = 2.0
    ; debt = 3500.0
    ; targetLeverage = 4.0}

// premise:
// the price of an asset should be the value of the collateral minus the debt divided by 
// the collateral minus the collateral based value of the debt
let excessCollateral lsp = lsp.collateral / lsp.targetLeverage
let excessCollateralValue lsp = lsp.debt / (lsp.targetLeverage-1.0)
let k lsp = (excessCollateralValue lsp) * (excessCollateral lsp) 

// o = (c d (l - 1) + d (l - 1) t - k l)/((l - 1) (c + t)) and (l - 1) (c + t)!=0 and l!=0
let outDebt lsp collateralIn = 
    let kStart = k lsp
    (lsp.collateral * lsp.debt * (lsp.targetLeverage - 1.0) + lsp.debt * (lsp.targetLeverage - 1.0) * collateralIn - kStart) 
    / ((lsp.targetLeverage - 1.0) * (lsp.collateral + collateralIn))

let price lsp = 
    {| excessCollateral = excessCollateral lsp
    ; excessCollateralValue = excessCollateralValue lsp
    ; price = (excessCollateralValue lsp) / (excessCollateral lsp)
    |}

// t = c - (k l)/((l - 1) (d + o)) and (l - 1) (d + o)!=0 and l!=0
let outCollateral lsp debtIn = 
    let kStart = k lsp
    let out = lsp.collateral - (kStart * lsp.targetLeverage) / ((lsp.targetLeverage - 1.0) * (lsp.debt + debtIn))
    {|out = out
    ; price = price lsp
    ; effectivePrice = debtIn / out|}

printf "collateral returned: %A\n" (outCollateral lsp 10.0)

// [price 1.0 1000.0 3.0;

// // it follows that:
// // * after a trade the price should have adjusted itself automatically in like with the trade's direction.
// // * after a buy of collateral : price (collateral-buy) (debt+out) > price collateral debt

// price (1.0 - 0.1) (1000.0 + 10.0) 3.0]


collateral returned: { effectivePrice = 5.622997508
  out = 1.77841089
  price = { excessCollateral = 0.5
            excessCollateralValue = 1166.666667
            price = 2333.333333 } }


In [ ]:
let getK x y = x * y // (x*y) = (x + in) * (y - out) solve for out 

let outB a b i = 
    let k = getK a b
    let out = (b - k/(a + i))
    let kNew = getK (a + i) (b - out)
    {|
        k = k;
        out = out;
        kNew = kNew;
        effectivePrice = out/i;
    |}

let outA a b i = outB b a i

[outB 20.0M 10.0M 3.0M
;outA 20.0M 10.0M 2.0M]


index,effectivePrice,k,kNew,out
0,0.4347826086956521739130434783,200.00,200.00000000000000000000000000,1.304347826086956521739130435
1,1.6666666666666666666666666665,200.00,200.00000000000000000000000000,3.333333333333333333333333333
